In [ ]:
import pandas as pd

In [ ]:
def preproceseaza_datele(df):
    # Se creează o copie a DataFrame-ului original pentru a nu-l modifica direct
    df = df.copy()

    # Curățare string-uri în toate coloanele de tip obiect:
    for col in df.columns:
        if df[col].dtype == object:
            # Elimină spațiile de la început/sfârșit și ghilimelele duble din șiruri
            df[col] = df[col].apply(lambda x: x.strip().replace('"', '') if isinstance(x, str) else x)
    
    # Transformă șirul de simptome separate prin virgulă într-o listă de simptome individuale
    df['Simptome'] = df['Simptome'].apply(
        lambda x: [s.strip() for s in x.split(',')] if isinstance(x, str) else []
    )
    
    return df

In [ ]:
def analizeaza_datele(df):
    """
    Afișează o analiză sumară a DataFrame-ului:
    - număr total de cazuri
    - tipuri de date pe coloane
    - număr de categorii unice
    - distribuția celor mai frecvente valori
    """

    print("="*50)
    print(" ANALIZA GENERALĂ A DATELOR")
    print("="*50)
    print(f" Total cazuri: {len(df)}")
    
    # Tipurile de date pentru fiecare coloană
    print("\n Tipuri de date pe coloane:")
    print("-"*40)
    print(df.dtypes)

    # Număr de valori unice per coloană
    print("\n Număr de categorii unice per coloană:")
    print("-"*40)
    for col in df.columns:
        if df[col].dtype == object or isinstance(df[col].iloc[0], list):
            if isinstance(df[col].iloc[0], list):
                # Aplatizează listele pentru a obține elementele unice
                flat_list = [item for sublist in df[col] for item in sublist]
                unique_items = set(flat_list)
                print(f" {col}: {len(unique_items)} categorii distincte (listă)")
            else:
                print(f" {col}: {df[col].nunique()} categorii distincte (text)")
        else:
            print(f" {col}: Numeric")

    # Cele mai frecvente 5 valori per coloană
    print("\n Distribuția valorilor cele mai frecvente (Top 5 per coloană):")
    print("-"*40)
    for col in df.columns:
        print(f"\n {col}:")
        print("-"*30)
        if df[col].dtype == object:
            print(df[col].value_counts().head())
        elif isinstance(df[col].iloc[0], list):
            flat_list = [item for sublist in df[col] for item in sublist]
            flat_series = pd.Series(flat_list)
            print(flat_series.value_counts().head())
        else:
            print(df[col].value_counts().head())

    print("\n Analiza completă.\n")


In [ ]:
df = pd.read_csv("../Definirea problemei/Data/peri_implantita_3000_cazuri.csv")

# Aplică preprocesarea
df_clean = preproceseaza_datele(df)

# Analizează datele curățate
analizeaza_datele(df_clean)

# Vizualizarea datelor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Distribuția cazurilor pozitive vs negative
# Reprezintă numărul total de cazuri pentru fiecare tip de diagnostic
sns.countplot(x='Diagnostic', hue='Diagnostic', data=df_clean, palette=['green', 'red'])
plt.title("Distribuția diagnosticului")
plt.xlabel("Diagnostic")
plt.ylabel("Număr cazuri")
plt.show()

# Distribuția vârstei în funcție de diagnostic
# Histograma vârstei, colorată în funcție de diagnostic, cu linie de densitate
sns.histplot(data=df_clean, x='Varsta', hue='Diagnostic', kde=True, bins=30, palette=['green', 'red'])
plt.title("Distribuția vârstei în funcție de diagnostic")
plt.xlabel("Vârstă")
plt.ylabel("Număr cazuri")
plt.show()

# Frecvența simptomelor în cazurile pozitive
# Extrage simptomele pentru cazurile cu diagnostic pozitiv și afișează top 10 cele mai frecvente
positive_symptoms = [s for sublist in df_clean[df_clean['Diagnostic'] == 'Peri-implantită']['Simptome'] for s in sublist]
positive_series = pd.Series(positive_symptoms).value_counts().head(10)

sns.barplot(x=positive_series.values, y=positive_series.index, hue=positive_series.index, palette='Reds')
plt.title("Top 10 simptome în cazuri pozitive")
plt.xlabel("Frecvență")
plt.ylabel("Simptom")
plt.show()

# Frecvența simptomelor în cazurile negative
# Extrage simptomele pentru cazurile cu diagnostic negativ și afișează top 10 cele mai frecvente
negative_symptoms = [s for sublist in df_clean[df_clean['Diagnostic'] == 'Fără peri-implantită']['Simptome'] for s in sublist]
negative_series = pd.Series(negative_symptoms).value_counts().head(10)

sns.barplot(x=negative_series.values, y=negative_series.index, hue=negative_series.index, palette='Greens')
plt.title("Top 10 simptome în cazuri negative")
plt.xlabel("Frecvență")
plt.ylabel("Simptom")
plt.show()

# Distribuția sexului în cazurile pozitive
# Compară numărul de cazuri pozitive pentru fiecare sex
sns.countplot(x='Sex', hue='Sex', data=df_clean[df_clean['Diagnostic'] == 'Peri-implantită'], palette="Set2")
plt.title("Distribuția pe sexe în cazuri pozitive")
plt.xlabel("Sex")
plt.ylabel("Număr cazuri")
plt.show()

# Relația dintre fumat și diagnostic
# Comparația numărului de cazuri în funcție de statutul de fumător
sns.countplot(x='Fumator', hue='Diagnostic', data=df_clean,
              palette={'Peri-implantită': 'red', 'Fără peri-implantită': 'green'})
plt.title("Fumat și diagnostic")
plt.xlabel("Fumător")
plt.ylabel("Număr cazuri")
plt.show()

# Relația dintre diabet și diagnostic
# Comparația numărului de cazuri în funcție de prezența diabetului
sns.countplot(x='Diabet', hue='Diagnostic', data=df_clean,
              palette={'Peri-implantită': 'red', 'Fără peri-implantită': 'green'})
plt.title("Diabet și diagnostic")
plt.xlabel("Diabet")
plt.ylabel("Număr cazuri")
plt.show()

# Distribuția numărului de ani de la implant în funcție de diagnostic
# Histograma densității în funcție de diagnostic
sns.histplot(data=df_clean, x='Ani_de_la_implant', hue='Diagnostic', kde=True,
             bins=20, palette={'Peri-implantită': 'red', 'Fără peri-implantită': 'green'},
             stat='density', common_norm=False)
plt.title("Ani de la implant în funcție de diagnostic")
plt.xlabel("Ani de la implant")
plt.ylabel("Densitate")
plt.show()

# Distribuția vârstei în funcție de diagnostic și sex
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_clean, x='Diagnostic', y='Varsta', hue='Sex',
            palette={'Masculin': 'skyblue', 'Feminin': 'lightpink'})
plt.title("Distribuția vârstei după diagnostic și sex")
plt.xlabel("Diagnostic")
plt.ylabel("Vârstă")
plt.legend(title='Sex')
plt.show()

print("afisare densitate")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


risk_table = pd.crosstab(index=[df_clean['Fumator'], df_clean['Diabet']],
                         columns=df_clean['Diagnostic'])

risk_table['Total'] = risk_table.sum(axis=1)
risk_table['Procentaj_Pozitiv'] = (risk_table['Peri-implantită'] / risk_table['Total']) * 100

heatmap_data = risk_table.reset_index().pivot(index='Diabet', columns='Fumator', values='Procentaj_Pozitiv')

plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_data, annot=True, cmap='Reds', fmt='.1f', linewidths=.5, cbar_kws={'label': 'Procentaj cazuri pozitive (%)'})
plt.title('Rata de Peri-implantită în funcție de Fumat și Diabet')
plt.xlabel('Statut Fumător')
plt.ylabel('Prezența Diabetului')
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
import sys
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


def incarca_datele(cale):
    df = pd.read_csv(cale)
    return df

def preproceseaza_datele(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].apply(lambda x: x.strip().replace('"', '') if isinstance(x, str) else x)
    df['Simptome'] = df['Simptome'].apply(lambda x: [s.strip() for s in x.split(',')] if isinstance(x, str) else [])
    return df


def extrage_caracteristici_imbunatatit(df):
    df = df.copy()
    df['Fumator'] = df['Fumator'].map({'Da': 1, 'Nu': 0})
    df['Diabet'] = df['Diabet'].map({'Da': 1, 'Nu': 0})
    df['Sex'] = df['Sex'].map({'F': 0, 'M': 1})
    df['Diagnostic'] = df['Diagnostic'].map({'Peri-implantită': 1, 'Fără peri-implantită': 0})
    df['Nr_Simptome'] = df['Simptome'].apply(len)

    """
    simptome_relevante = [
        'sângerare', 'halitoză', 'inflamație severă', 'retragere gingivală',
        'durere', 'mobilitate implant', 'niciun simptom', 'inflamație gingivală ușoară'
    ]
    """

    simptome_relevante = [
        'sângerare', 'halitoză', 'edem', 'durere' 'inflamație severă', 'retragere gingivală', 'durere la masticație', 'mobilitate implant', 'pierderea osului marginal','sângerare ușoară'
    ]
    for simptom in simptome_relevante:
        df[f"sympt_{simptom}"] = df['Simptome'].apply(
            lambda lista, s=simptom: int(s in lista)
        )

    return df[['Varsta', 'Ani_de_la_implant', 'Fumator', 'Diabet', 'Sex', 'Nr_Simptome'] +
               [f"sympt_{s}" for s in simptome_relevante] + ['Diagnostic']]

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
import pandas as pd

def antreneaza_model(df_feat):
    zgomot_idx = df_feat.sample(frac=0.1, random_state=1).index
    df_feat.loc[zgomot_idx, 'Nr_Simptome'] += 1  

    X = df_feat.drop("Diagnostic", axis=1)
    y = df_feat["Diagnostic"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    model = RandomForestClassifier(
        n_estimators=50,   
        max_depth=1,
        max_features=3,    
        random_state=42
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print("\n=== METRICI DE PERFORMANȚĂ ===")
    print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
    print("Precision:", round(precision_score(y_test, y_pred, zero_division=1), 3))
    print("Recall:", round(recall_score(y_test, y_pred), 3))
    print("F1 Score:", round(f1_score(y_test, y_pred), 3))

    print("\nMatrice de confuzie:")
    print(confusion_matrix(y_test, y_pred))

    print("\nRaport complet:")
    print(classification_report(y_test, y_pred, zero_division=1))

    return model, accuracy_score(y_test, y_pred)


def evalueaza_pe_toate_cazurile(model, df_feat):
    X_all = df_feat.drop("Diagnostic", axis=1)
    y_real = df_feat["Diagnostic"]
    y_pred = model.predict(X_all)

    print("\n=== EVALUARE PE ÎNTREG SETUL DE DATE ===")
    print(f"Acuratețe generală: {round(accuracy_score(y_real, y_pred) * 100, 2)}%")
    print("Precision:", round(precision_score(y_real, y_pred, zero_division=1), 3))
    print("Recall:", round(recall_score(y_real, y_pred), 3))
    print("F1 Score:", round(f1_score(y_real, y_pred), 3))
    print("\nMatrice de confuzie:")
    print(confusion_matrix(y_real, y_pred))
    print("\nRaport complet:")
    print(classification_report(y_real, y_pred, zero_division=1))

if __name__ == "__main__":
    cale_fisier = "../Definirea problemei/Data/peri_implantita_3000_cazuri.csv"  
    df = incarca_datele(cale_fisier)
    df = preproceseaza_datele(df)
    df_feat = extrage_caracteristici_imbunatatit(df)

    model, acc_score = antreneaza_model(df_feat)
    evalueaza_pe_toate_cazurile(model, df_feat)
    print(f"Acuratețe calculată: {acc_score}")

In [ ]:
import random
import pandas as pd

def transforma_input_random(columns):
    #am luat primele 10 cele mai intalnite simptome in cazuri pozitive
    simptome = [
        'sângerare', 'halitoză', 'edem', 'durere', 'inflamație severă', 'retragere gingivală', 'durere la masticație', 'mobilitate implant', 'pierderea osului marginal','niciun simptom','inflamație gingivală ușoară','ușoară sângerare','ușoară roșeață gingivală'
    ]
    simptome_ales = random.sample(simptome, k=random.randint(1, 3))

    row = {
        'Varsta': random.randint(18, 99),
        'Ani_de_la_implant': random.randint(0, 50),
        'Fumator': random.randint(0, 1),
        'Diabet': random.randint(0, 1),
        'Sex': random.randint(0, 1),
        'Nr_Simptome': len(simptome_ales)
    }

    for s in simptome:
        row[f"sympt_{s}"] = int(s in simptome_ales)

    return pd.DataFrame([row], columns=columns), simptome_ales


for i in range(5):
    sample, simptome_generate = transforma_input_random(model.feature_names_in_)
    rezultat = model.predict(sample)[0]
    rezultat_text = "PERI-IMPLANTITĂ" if rezultat == 1 else "FĂRĂ PERI-IMPLANTITĂ"

    row = sample.iloc[0].copy().astype(object)
    row["Fumator"] = "Da" if row["Fumator"] == 1 else "Nu"
    row["Diabet"] = "Da" if row["Diabet"] == 1 else "Nu"
    row["Sex"] = "M" if row["Sex"] == 1 else "F"

    print(f"\n=== CAZ RANDOM #{i+1} ===")
    print(row.drop([col for col in row.index if col.startswith("sympt_")]).to_frame(name="Valoare"))
    print("Simptome alese:", ', '.join(simptome_generate))
    print(">>> Predicție:", rezultat_text)


# Exemplu interactiune utilizator

In [ ]:
def ruleaza_formular(model, simptome_relevante):

    # Validare pentru varsta
    while True:
        try:
            varsta = int(input("Vârsta (18-99): "))
            if 18 <= varsta <= 99:
                break
            else:
                continue
        except ValueError:
            print("Introduceți un număr întreg valid.")

    # Validare pentru sex
    while True:
        sex = input("Sex (M/F): ").strip().upper()
        if sex in ['M', 'F']:
            break
        else:
            continue

    # Validare pentru fumator
    while True:
        fumator = input("Fumător (Da/Nu): ").strip().capitalize()
        if fumator in ['Da', 'Nu']:
            break
        else:
            continue

    # Validare pentru diabet
    while True:
        diabet = input("Diabet (Da/Nu): ").strip().capitalize()
        if diabet in ['Da', 'Nu']:
            break
        else:
            continue

    # Validare pentru ani de la implant
    while True:
        try:
            ani_de_la_implant = int(input("Ani de la implant trebuie sa fie o valuare intre (0-50): "))
            if 0 <= ani_de_la_implant <= 50:
                break
            else:
                continue
        except ValueError:
            print("Introduceți un număr întreg valid.")

    #Select options (combobox) pt simptome de implementat ulterior
    simptome_text = input("Simptome (separate prin virgulă): ").strip()
    simptome = [s.strip() for s in simptome_text.split(',') if s.strip()]

    # Construim un DataFrame cu o singură intrare pentru model
    import pandas as pd

    data = {
        'Varsta': [varsta],
        'Ani_de_la_implant': [ani_de_la_implant],
        'Fumator': [1 if fumator == 'Da' else 0],
        'Diabet': [1 if diabet == 'Da' else 0],
        'Sex': [1 if sex == 'M' else 0],
        'Nr_Simptome': [len(simptome)],
    }

    for simptom in simptome_relevante:
        data[f"sympt_{simptom}"] = [1 if simptom in simptome else 0]

    df_input = pd.DataFrame(data)

    # Prezice diagnosticul
    pred = model.predict(df_input)[0]

    print("Varsta: ",varsta)
    print("Sex: ",sex)
    print("Fumator: ",fumator)
    print("Diabet: ",diabet)
    print("Ani de la implant: ",ani_de_la_implant)
    print("Simptome: ", simptome_text)
    if pred == 1:
        return "DIAGNOSTIC: Peri-implantită (pozitiv)."
    else:
        return "DIAGNOSTIC: Fără peri-implantită (negativ)."

In [ ]:
simptome_relevante = [
        'sângerare', 'halitoză', 'edem', 'durere' 'inflamație severă', 'retragere gingivală', 'durere la masticație', 'mobilitate implant', 'pierderea osului marginal'
    ]
# rulam formularul cu modelul si simpotmele relevante
rezultat = ruleaza_formular(model, simptome_relevante)
print(rezultat)